# 데이터 전처리 진행

In [1]:
#!pip install python-mecab-ko pyLDAvis

In [2]:
import numpy as np
import pandas as pd
import warnings # 경고 메시지 무시

In [3]:
df = pd.read_csv('../datasets/oliveyoung_wemake_data.csv')
df[:2]

,Unnamed: 0,product_name,review_name,review_content,review_date,review_rating
0,0,"[주현영, 이나연 찐템]딥트 3일 피치(맛)번 30정 (30일분)",보리오빠,없으면 안될 최고의 diet템! 피치번을 추천합니당 ㅎㅎ 사진엔 없지만 4통째 먹고...,2023.08.27,5점만점에 5점
1,1,"[주현영, 이나연 찐템]딥트 3일 피치(맛)번 30정 (30일분)",바보바니,알약 통통이하트 넘예뻐요. 자기전 먹고 효과있으면 얼마나 좋을까요 ㅜㅜ 나이들수록 ...,2023.09.14,5점만점에 5점


### 결측치 처리 

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4589 entries, 0 to 4588
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      4589 non-null   int64 
 1   product_name    4589 non-null   object
 2   review_name     4570 non-null   object
 3   review_content  3900 non-null   object
 4   review_date     4463 non-null   object
 5   review_rating   4423 non-null   object
dtypes: int64(1), object(5)
memory usage: 215.2+ KB


In [5]:
df.isnull().sum()

Unnamed: 0          0
product_name        0
review_name        19
review_content    689
review_date       126
review_rating     166
dtype: int64

In [6]:
df_dropna = df.dropna()

In [7]:
df_dropna.isnull().sum()

Unnamed: 0        0
product_name      0
review_name       0
review_content    0
review_date       0
review_rating     0
dtype: int64

In [8]:
df_dropna[3:]

,Unnamed: 0,product_name,review_name,review_content,review_date,review_rating
3,3,"[주현영, 이나연 찐템]딥트 3일 피치(맛)번 30정 (30일분)",히드돈이,먹으면 달달한 사탕맛이 나는데\n2-3초 빨아 먹으면 달달구리는 다 녹아서 맛없는 ...,2023.09.04,5점만점에 5점
4,4,"[주현영, 이나연 찐템]딥트 3일 피치(맛)번 30정 (30일분)",짱구덕휴,효과는 모르겠지만 덜찌는것 같아요\n알약이 이쁘고 입안에 들어갔을때 역하지않아서 좋습니다,2023.09.04,5점만점에 5점
5,5,"[주현영, 이나연 찐템]딥트 3일 피치(맛)번 30정 (30일분)",하늘색벽지,효과가 가장 있는 제품인 것 같아요!\n너무 좋습니다!\n세일기간만 기다리고 있습니...,2023.08.25,5점만점에 5점
6,6,"[주현영, 이나연 찐템]딥트 3일 피치(맛)번 30정 (30일분)",IMIN,연달아서 먹어야 효과를 알텐데 약 때문에 꾸준히 못먹어봐서 잘 모르겠어요 그래도 혹...,2023.08.25,5점만점에 5점
7,7,"[주현영, 이나연 찐템]딥트 3일 피치(맛)번 30정 (30일분)",수아짱,식단만으로 부족해서 도움받으려고 구매했어요 자기 전 한알 신기하네요,2023.09.10,5점만점에 4점
...,...,...,...,...,...,...
4583,1367,[4+1] 락토페린 산양유 초유 단백질 분말 스틱 (2g x 30포) / 네덜란드산...,oke*******,친정엄마께 드릴려고 삽니다.산양유 드셔보고싶어하셔서 좋은기회인것같아요,2023.01.03,5.0
4584,1368,[4+1] 락토페린 산양유 초유 단백질 분말 스틱 (2g x 30포) / 네덜란드산...,whg******,산양유 단백질 광고보고 어떤지 구매해봤어요. 많이 달지도 않고 먹기 편하고 좋네요.^^,2022.12.14,4.0
4585,1369,[4+1] 락토페린 산양유 초유 단백질 분말 스틱 (2g x 30포) / 네덜란드산...,app*****,꾸준히 먹고는있는데 어디가 좋아지는건지..맛있어서 매일 챙겨먹고는 있어요,2023.01.31,4.0
4586,1370,테이크핏 맥스 바나나맛 250ml X 24입,jih******,뭔가 끈적거려서 보니 제품 하나가 찢어져 있었어요.,2023.09.20,3.0


## 자연어 전처리 

### 형태소 분석

In [9]:
from konlpy.tag import Okt
okt = Okt()

In [10]:
content = list(df_dropna['review_content']) 

In [11]:
sentence = ' '.join(content) # 한줄로 만들기 

In [12]:
word_tokens = okt.morphs(sentence)

In [13]:
word_pos = okt.pos(sentence)

### 불필요한 심볼 없애기 

In [14]:
# 쉼표, 엔터 등등의 값 대체 
# filtered_content = df_dropna['review_content'].replace('.', '').replace(',','').replace("'","").replace('·', ' ').replace('=','').replace('\n','').replace('\\n',' ').replace('\\n\\n','  ')
# list(filtered_content)

### 불용어 처리 

In [15]:
stop_words = pd.read_csv('../datasets/koreanStopwords.txt', header=None, encoding ='utf-8')
#stop_words = stop_words[0]
stop_words= stop_words[0].values

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/koreanStopwords.txt'

In [ ]:
one_char_keyword = pd.read_csv('../datasets/one_char_list.txt', header=None, encoding ='utf-8')
one_char_keyword = one_char_keyword[0].values

In [ ]:
from konlpy.tag import Okt
okt = Okt()
from mecab import MeCab
mecab = MeCab()

In [ ]:
# 토크나이징 함수 정의
def remove_stopword(raw, pos=["Noun","Alpha","Verb","Number","Adjective"], stopword=stop_words):
    tokens = [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅏ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword

    ]
    return ' '.join(tokens)  # 결과 리스트를 문자열로 결합하여 반환

In [ ]:
# 토크나이징 함수 정의(불용어, 한글자 처리)
def remove_stopword(raw, pos=["Noun","Alpha","Verb","Number","Adjective"], stopword=stop_words):
    tokens = []
    for word, tag in okt.pos(
        raw, 
        norm=True,   # normalize 그랰ㅋㅏ -> 그래ㅋㅋ
        stem=True    # stemming 바뀌나->바뀌다
    ):
        if len(word) > 1 and tag in pos and word not in stopword:
            tokens.append(word)
        elif len(word) == 1 and word in list(one_char_keyword):
            tokens.append(word)
    return ' '.join(tokens)  # 결과 리스트를 문자열로 결합하여 반환

In [ ]:
## 함수적용 
df_dropna['tokenized_review'] = df_dropna['review_content'].apply(remove_stopword)
df_dropna[:2]

In [ ]:
# 확인용 
test_sentence = "이 문장은 불용어 처리가 되어야 합니다. 가다 하다 있다 되다 먹다"
tokens_test = remove_stopword(test_sentence)
print(tokens_test)

### 빈도수 계산 

In [ ]:
from collections import Counter
from wordcloud import WordCloud

In [ ]:
sentence_tokenized= ' '.join(list(df_dropna['tokenized_review'])) # 한줄로 만들기 

In [ ]:
sentence_split = sentence_tokenized.split()

In [ ]:
# counter를 이용해서 중복 여부 count
count = Counter(sentence_split)

# 한글 폰트 경로 설정 (Windows 환경에 맞게 경로 수정)
font_path = 'C:/Windows/Fonts/malgun.ttf'

wordCloud = WordCloud(font_path=font_path, background_color='white') #인스턴스화 
wordCloud.generate_from_frequencies(count)

In [ ]:
# count 
wordCloud.words_ 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
countVectorizer = CountVectorizer() # 인스턴스화
countVectorizer.fit(count)

In [ ]:
countVectorizer.vocabulary_  #텍스트 데이터를 벡터화  #단어-인덱스 매핑
# 특정 단어가 벡터의 어떤 위치에 해당하는지를 파악하는 데 도움

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(wordCloud)
plt.axis('off') 
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


## 막대그래프 그리기 

### 유사어 대치 

In [ ]:
df_replace_list = pd.read_csv('../datasets/replace_list.csv')
df_replace_list[:3]

In [ ]:
def replace_word(tokenized_review):
    replaced_review = str()
    for i in range(len(df_replace_list['before_replacement'])):
        try:
          # 치환할 단어가 있는 경우에만 데이터 치환 수행
          if df_replace_list['before_replacement'][i] in tokenized_review:
            tokenized_review = tokenized_review.replace(df_replace_list['before_replacement'][i], df_replace_list['after_replacement'][i])
        except Exception as e:
            print(f"Error 발생 / 에러명: {e}")
    return tokenized_review

In [ ]:
## 확인용 
replace_word('어트 stess 살이 빠지다 피치 달달 단맛')

In [ ]:
df_dropna['replaced_review'] = df_dropna['tokenized_review'].apply(replace_word)
df_dropna[:2]

## 긍/부정 댓글 분리 

In [ ]:
import re
df_dropna['rating'] = df_dropna['review_rating'].str.extract(r'(\s\d(?=점))|(\d.\d)').apply(lambda x: ''.join(x.dropna()), axis=1)
df_dropna[:2]

In [ ]:
##  'rating' 열을 숫자로 변환
df_dropna['rating'] = pd.to_numeric(df_dropna['rating'])

In [ ]:
condition_positive =  df_dropna['rating'] >= 4 # 긍정
condition_negative = df_dropna['rating'] < 3  # 부정

In [ ]:
df_review_positive = df_dropna[condition_positive]
df_review_negative= df_dropna[condition_negative]

In [ ]:
df_review_positive[:2]

In [ ]:
df_review_negative[:2]

# LDA 분석

In [ ]:
from gensim.corpora import Dictionary

In [ ]:
dictionary = Dictionary(df_review_negative[['replaced_review']].values)
dictionary

In [ ]:
dictionary.doc2bow(df_review_negative[['replaced_review']].values[3])

In [ ]:
# dictionary에 의한 한글 문장을 벡터화 변환
corpus_list = list()
for sentence in df_review_negative[['replaced_review']].values :
    vectors = dictionary.doc2bow(sentence)
    corpus_list.append(vectors)

In [ ]:
df_review_negative['Vectorization'] = corpus_list
df_review_negative[:3]

## 토픽 잡기 

In [ ]:
from gensim.models import LdaModel

In [ ]:
lda_model = LdaModel(corpus=corpus_list, id2word=dictionary, num_topics=3)

In [ ]:
lda_model.print_topics(num_words=4)

## 토픽 분석 

### LDA 시각화 

In [ ]:
#%pip install pyLDAvis

In [ ]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

In [ ]:
result_visualized = pyLDAvis.gensim_models.prepare(lda_model, corpus_list, dictionary)
pyLDAvis.display(result_visualized)

In [ ]:
pyLDAvis.save_html(result_visualized, './result_visualized_negative.html')

## LDA 토픽 모델링 

#### 하이퍼파라미터 튜닝

In [ ]:
NUM_TOPICS = 10 # 토픽 개수는 하이퍼파라미터
# passes: 딥러닝에서 Epoch와 같은 개념으로, 전체 corpus로 모델 학습 횟수 결정
PASSES = 15

### 모델 학습 함수 

In [ ]:
from gensim import corpora # 단어 빈도수 계산 패키지
import gensim # LDA 모델 활용 목적
import pyLDAvis.gensim_models # LDA 시각화용 패키지

In [ ]:
dictionary = corpora.Dictionary(df_dropna['replaced_review'].apply(lambda x: x.split()).values)
corpus = [dictionary.doc2bow(review.split()) for review in df_dropna['replaced_review'].values]
# LDA 모델 학습
model = gensim.models.ldamodel.LdaModel(corpus, 
                                            num_topics=NUM_TOPICS, 
                                            id2word=dictionary, 
                                            passes=PASSES)

In [ ]:
def lda_modeling(review_prep):
    # 단어 인코딩 및 빈도수 계산
    dictionary = corpora.Dictionary(df_dropna['replaced_review'].apply(lambda x: x.split()).values)
    corpus = [dictionary.doc2bow(review.split()) for review in df_dropna['replaced_review'].values]
    # LDA 모델 학습
    model = gensim.models.ldamodel.LdaModel(corpus, 
                                            num_topics=NUM_TOPICS, 
                                            id2word=dictionary, 
                                            passes=PASSES)
    return model, corpus, dictionary

### 토픽별 단어 구성 출력 함수 

In [ ]:
def print_topic_prop(topics, RATING):
    topic_values = []
    for topic in topics:
        topic_value = topic[1]
        topic_values.append(topic_value)
    topic_prop = pd.DataFrame({"topic_num" : list(range(1, NUM_TOPICS + 1)), "word_prop": topic_values})
    #topic_prop.to_excel('./result/topic_prop_' + RATING +  '.xlsx')
    display(topic_prop)

### 시각화 함수 

In [ ]:
def lda_visualize(model, corpus, dictionary, RATING):
    pyLDAvis.enable_notebook()
    result_visualized = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
    pyLDAvis.display(result_visualized)
    # 시각화 결과 저장
    # RESULT_FILE = './result/lda_result_' + RATING + '.html'
    # pyLDAvis.save_html(result_visualized, RESULT_FILE)

### 긍정정 리뷰 토픽 모델링 

In [ ]:
model, corpus, dictionary = lda_modeling(df_review_positive)
NUM_WORDS = 10

In [ ]:
from gensim import corpora 

In [ ]:
RATING = 'pos'
topics = model.print_topics(num_words = NUM_WORDS)
print_topic_prop(topics, RATING)

In [ ]:
# lda_visualize(model, corpus, dictionary, RATING)

### 부정적 리뷰 토픽 모델링

In [ ]:
model, corpus, dictionary = lda_modeling(df_review_negative)
NUM_WORDS = 10

In [ ]:
RATING = 'neg'
topics = model.print_topics(num_words = NUM_WORDS)
print_topic_prop(topics, RATING)

In [ ]:
lda_visualize(model, corpus, dictionary, RATING)

In [ ]:
df_dropna['replaced_review']

In [ ]:
df_dropna['replaced_review_2'] = df_dropna['replaced_review'].apply(lambda x: [x])

In [ ]:
replaced_review_2 = df_dropna['replaced_review_2']

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(replaced_review_2)
corpus = [dictionary.doc2bow(text) for text in replaced_review_2]

In [ ]:
dictionary = corpora.Dictionary(df_dropna['replaced_review'].apply(lambda x: x.split()).values)
corpus = [dictionary.doc2bow(review.split()) for review in df_dropna['replaced_review'].values]
# LDA 모델 학습
model = gensim.models.ldamodel.LdaModel(corpus, 
                                            num_topics=NUM_TOPICS, 
                                            id2word=dictionary, 
                                            passes=PASSES)


In [ ]:
dictionary[25]

In [ ]:
corpus[1]

In [ ]:
len(dictionary)

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)